In [ ]:
!pip install hdbscan
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.2 MB/s eta 0:00:00
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Successfully uninstalled Cython-3.0.10


In [1]:
import pandas as pd

# URLs das bases de dados
wine_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'
breast_cancer_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data'
abalone_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data'
ecoli_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/ecoli/ecoli.data'

wine = pd.read_csv(wine_url, sep=';')
breast_cancer = pd.read_csv(breast_cancer_url, header=None)
abalone = pd.read_csv(abalone_url, header=None)
ecoli = pd.read_csv(ecoli_url, header=None, delim_whitespace=True)

print("Wine Quality Dataset: ok")
#print(wine.head())

print("Breast Cancer Wisconsin (Original) Dataset: ok")
#print(breast_cancer.head())

print("Abalone Dataset: ok")
#print(abalone.head())

print("Ecoli Dataset: ok")
#print(ecoli.head())

/tmp/ipykernel_17584/681074065.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  ecoli = pd.read_csv(ecoli_url, header=None, delim_whitespace=True)


Wine Quality Dataset: ok
Breast Cancer Wisconsin (Original) Dataset: ok
Abalone Dataset: ok
Ecoli Dataset: ok


In [2]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Verificar valores ausentes
#print("Valores ausentes no Wine Quality Dataset:")
#print(wine.isnull().sum())

# Remover outliers usando o IQR (Interquartile Range)
Q1 = wine.quantile(0.25)
Q3 = wine.quantile(0.75)
IQR = Q3 - Q1
wine_clean = wine[~((wine < (Q1 - 1.5 * IQR)) | (wine > (Q3 + 1.5 * IQR))).any(axis=1)]

# Normalizar os dados
scaler = StandardScaler()
wine_scaled = scaler.fit_transform(wine_clean.drop(columns=['quality']))
wine_clean_scaled = pd.DataFrame(wine_scaled, columns=wine_clean.columns[:-1])

#print("\nWine Quality Dataset após pré-processamento:")
#print(wine_clean_scaled.head())
print('pré-processamento de Wine concluído!')


# Atribuir nomes às colunas
breast_cancer.columns = ['ID', 'Diagnosis'] + [f'Feature_{i}' for i in range(1, 31)]

# Verificar valores ausentes
#print("Valores ausentes no Breast Cancer Wisconsin Dataset:")
#print(breast_cancer.isnull().sum())

# Remover a coluna de ID e converter a coluna 'Diagnosis' para numérico
breast_cancer_clean = breast_cancer.drop(columns=['ID'])
breast_cancer_clean['Diagnosis'] = breast_cancer_clean['Diagnosis'].apply(lambda x: 1 if x == 'M' else 0)

# Remover outliers usando o IQR
Q1 = breast_cancer_clean.quantile(0.25)
Q3 = breast_cancer_clean.quantile(0.75)
IQR = Q3 - Q1
breast_cancer_clean = breast_cancer_clean[~((breast_cancer_clean < (Q1 - 1.5 * IQR)) | (breast_cancer_clean > (Q3 + 1.5 * IQR))).any(axis=1)]

# Normalizar os dados
scaler = StandardScaler()
breast_cancer_scaled = scaler.fit_transform(breast_cancer_clean.drop(columns=['Diagnosis']))
breast_cancer_clean_scaled = pd.DataFrame(breast_cancer_scaled, columns=breast_cancer_clean.columns[1:])

#print("\nBreast Cancer Wisconsin Dataset após pré-processamento:")
#print(breast_cancer_clean_scaled.head())
print('pré-processamento de Breast Cancer concluído!')


# Atribuir nomes às colunas
abalone.columns = ['Sex', 'Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight', 'Rings']

# Verificar valores ausentes
#print("Valores ausentes no Abalone Dataset:")
#print(abalone.isnull().sum())

# Converter 'Sex' para numérico
abalone_clean = abalone.copy()
abalone_clean['Sex'] = abalone_clean['Sex'].map({'M': 1, 'F': 0, 'I': 2})

# Remover outliers usando o IQR
Q1 = abalone_clean.quantile(0.25)
Q3 = abalone_clean.quantile(0.75)
IQR = Q3 - Q1
abalone_clean = abalone_clean[~((abalone_clean < (Q1 - 1.5 * IQR)) | (abalone_clean > (Q3 + 1.5 * IQR))).any(axis=1)]

# Normalizar os dados
scaler = StandardScaler()
abalone_scaled = scaler.fit_transform(abalone_clean.drop(columns=['Rings']))
abalone_clean_scaled = pd.DataFrame(abalone_scaled, columns=abalone_clean.columns[:-1])

#print("\nAbalone Dataset após pré-processamento:")
#print(abalone_clean_scaled.head())
print('pré-processamento de Abalone concluído!')


# Atribuir nomes às colunas
ecoli.columns = ['Sequence_name', 'mcg', 'gvh', 'lip', 'chg', 'aac', 'alm1', 'alm2', 'Class']

# Verificar tipos de dados de cada coluna
#print("Tipos de dados das colunas:")
#print(ecoli.dtypes)

# Remover a coluna de nome de sequência, pois não é útil para a análise
ecoli_clean = ecoli.drop(columns=['Sequence_name'])

# Converter todas as colunas exceto 'Class' para numérico
ecoli_clean.iloc[:, :-1] = ecoli_clean.iloc[:, :-1].apply(pd.to_numeric, errors='coerce')

# Verificar valores ausentes após conversão
#print("\nValores ausentes após conversão para numérico:")
#print(ecoli_clean.isnull().sum())

# Remover linhas com valores ausentes (resultantes da conversão para numérico)
ecoli_clean = ecoli_clean.dropna()

# Verificar novamente os tipos de dados após a limpeza
#print("\nTipos de dados após a limpeza:")
#print(ecoli_clean.dtypes)

# Verificar novamente os valores únicos nas colunas para garantir que são numéricos
#print("\nValores únicos nas colunas após a limpeza:")
#for col in ecoli_clean.columns[:-1]:  # Exceto 'Class'
#    print(f"{col}: {ecoli_clean[col].unique()}")

# Remover a coluna 'Class' antes de remover outliers e normalizar os dados
ecoli_features = ecoli_clean.drop(columns=['Class'])

# Remover outliers usando o IQR
#print("\nRemoção de outliers:")
try:
    Q1 = ecoli_features.quantile(0.25)
    Q3 = ecoli_features.quantile(0.75)
    IQR = Q3 - Q1
    ecoli_features = ecoli_features[~((ecoli_features < (Q1 - 1.5 * IQR)) | (ecoli_features > (Q3 + 1.5 * IQR))).any(axis=1)]
    #print("Outliers removidos com sucesso.")
except Exception as e:
    print(f"Erro ao remover outliers: {e}")

# Normalizar os dados
try:
    scaler = StandardScaler()
    ecoli_scaled = scaler.fit_transform(ecoli_features)
    ecoli_clean_scaled = pd.DataFrame(ecoli_scaled, columns=ecoli_features.columns)
    #print("\nEcoli Dataset após pré-processamento:")
    #print(ecoli_clean_scaled.head())
except Exception as e:
    print(f"Erro ao normalizar os dados: {e}")

print('pré-processamento de Ecoli concluído!')

pré-processamento de Wine concluído!
pré-processamento de Breast Cancer concluído!
pré-processamento de Abalone concluído!
pré-processamento de Ecoli concluído!


In [3]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Função para executar K-means e calcular a pontuação de Silhouette
def run_kmeans(data, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
    labels = kmeans.fit_predict(data)
    silhouette_avg = silhouette_score(data, labels)
    return labels, silhouette_avg

# Número de clusters para cada dataset
clusters = {
    'wine': 6,
    'breast_cancer': 2,
    'abalone': 3,
    'ecoli': 8
}

# Wine Quality Dataset
wine_kmeans_labels, wine_silhouette = run_kmeans(wine_clean_scaled, clusters['wine'])
print(f"Wine Quality Dataset - K-Means Silhouette Score: {wine_silhouette}")

# Breast Cancer Wisconsin Dataset
breast_cancer_kmeans_labels, breast_cancer_silhouette = run_kmeans(breast_cancer_clean_scaled, clusters['breast_cancer'])
print(f"Breast Cancer Wisconsin Dataset - K-Means Silhouette Score: {breast_cancer_silhouette}")

# Abalone Dataset
abalone_kmeans_labels, abalone_silhouette = run_kmeans(abalone_clean_scaled, clusters['abalone'])
print(f"Abalone Dataset - K-Means Silhouette Score: {abalone_silhouette}")

# Ecoli Dataset
ecoli_kmeans_labels, ecoli_silhouette = run_kmeans(ecoli_clean_scaled, clusters['ecoli'])
print(f"Ecoli Dataset - K-Means Silhouette Score: {ecoli_silhouette}")


from sklearn.cluster import DBSCAN

# Função para executar DBSCAN e calcular a pontuação de Silhouette
def run_dbscan(data, eps, min_samples):
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(data)
    # Filtrar clusters para evitar valores -1 (ruído) no cálculo do coeficiente de silhueta
    if len(set(labels)) > 1:
        silhouette_avg = silhouette_score(data, labels)
    else:
        silhouette_avg = -1
    return labels, silhouette_avg

# Parâmetros do DBSCAN (por exemplo, eps=0.5, min_samples=5)
dbscan_params = {
    'eps': 0.5,
    'min_samples': 5
}

# Wine Quality Dataset
wine_dbscan_labels, wine_dbscan_silhouette = run_dbscan(wine_clean_scaled, **dbscan_params)
print(f"\nWine Quality Dataset - DBSCAN Silhouette Score: {wine_dbscan_silhouette}")

# Breast Cancer Wisconsin Dataset
breast_cancer_dbscan_labels, breast_cancer_dbscan_silhouette = run_dbscan(breast_cancer_clean_scaled, **dbscan_params)
print(f"Breast Cancer Wisconsin Dataset - DBSCAN Silhouette Score: {breast_cancer_dbscan_silhouette}")

# Abalone Dataset
abalone_dbscan_labels, abalone_dbscan_silhouette = run_dbscan(abalone_clean_scaled, **dbscan_params)
print(f"Abalone Dataset - DBSCAN Silhouette Score: {abalone_dbscan_silhouette}")

# Ecoli Dataset
ecoli_dbscan_labels, ecoli_dbscan_silhouette = run_dbscan(ecoli_clean_scaled, **dbscan_params)
print(f"Ecoli Dataset - DBSCAN Silhouette Score: {ecoli_dbscan_silhouette}")


import hdbscan

# Função para executar HDBSCAN e calcular a pontuação de Silhouette
def run_hdbscan(data, min_cluster_size):
    hdb = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size)
    labels = hdb.fit_predict(data)
    # Filtrar clusters para evitar valores -1 (ruído) no cálculo do coeficiente de silhueta
    if len(set(labels)) > 1:
        silhouette_avg = silhouette_score(data, labels)
    else:
        silhouette_avg = -1
    return labels, silhouette_avg

# Parâmetro do HDBSCAN (por exemplo, min_cluster_size=10)
hdbscan_params = {
    'min_cluster_size': 10
}

# Wine Quality Dataset
wine_hdbscan_labels, wine_hdbscan_silhouette = run_hdbscan(wine_clean_scaled, **hdbscan_params)
print(f"\nWine Quality Dataset - HDBSCAN Silhouette Score: {wine_hdbscan_silhouette}")

# Breast Cancer Wisconsin Dataset
breast_cancer_hdbscan_labels, breast_cancer_hdbscan_silhouette = run_hdbscan(breast_cancer_clean_scaled, **hdbscan_params)
print(f"Breast Cancer Wisconsin Dataset - HDBSCAN Silhouette Score: {breast_cancer_hdbscan_silhouette}")

# Abalone Dataset
abalone_hdbscan_labels, abalone_hdbscan_silhouette = run_hdbscan(abalone_clean_scaled, **hdbscan_params)
print(f"Abalone Dataset - HDBSCAN Silhouette Score: {abalone_hdbscan_silhouette}")

# Ecoli Dataset
ecoli_hdbscan_labels, ecoli_hdbscan_silhouette = run_hdbscan(ecoli_clean_scaled, **hdbscan_params)
print(f"Ecoli Dataset - HDBSCAN Silhouette Score: {ecoli_hdbscan_silhouette}")

Wine Quality Dataset - K-Means Silhouette Score: 0.11728840076300899
Breast Cancer Wisconsin Dataset - K-Means Silhouette Score: 0.33003687536221243
Abalone Dataset - K-Means Silhouette Score: 0.38629404359347685
Ecoli Dataset - K-Means Silhouette Score: 0.2120619776112445

Wine Quality Dataset - DBSCAN Silhouette Score: -0.31524590381029344
Breast Cancer Wisconsin Dataset - DBSCAN Silhouette Score: -1
Abalone Dataset - DBSCAN Silhouette Score: -0.09684161458619917
Ecoli Dataset - DBSCAN Silhouette Score: -0.1882033086586728

Wine Quality Dataset - HDBSCAN Silhouette Score: -0.24052857013966075
Breast Cancer Wisconsin Dataset - HDBSCAN Silhouette Score: -1
Abalone Dataset - HDBSCAN Silhouette Score: 0.14707927234201587
Ecoli Dataset - HDBSCAN Silhouette Score: 0.19617520783022557


In [4]:
from sklearn.metrics import adjusted_rand_score

# Supondo que temos os rótulos verdadeiros
wine_labels = wine_clean['quality'].values  # Usando os dados limpos
breast_cancer_labels = breast_cancer_clean['Diagnosis'].values  # Usando a coluna 'Diagnosis'
abalone_labels = abalone_clean['Rings'].values  # Usando a coluna 'Rings'
ecoli_labels = ecoli_clean['Class'].values  # Usando a coluna 'Class'

# Certificar que os índices dos rótulos correspondem aos dados limpos
wine_labels = wine_labels[:len(wine_kmeans_labels)]
breast_cancer_labels = breast_cancer_labels[:len(breast_cancer_kmeans_labels)]
abalone_labels = abalone_labels[:len(abalone_kmeans_labels)]
ecoli_labels = ecoli_labels[:len(ecoli_kmeans_labels)]

# Função para calcular o ARI
def calculate_ari(true_labels, predicted_labels):
    return adjusted_rand_score(true_labels, predicted_labels)

# Calcular ARI para cada algoritmo e dataset
print("Adjusted Rand Index (ARI):")

# K-means
print(f"Wine Quality Dataset - K-means ARI: {calculate_ari(wine_labels, wine_kmeans_labels)}")
print(f"Breast Cancer Wisconsin Dataset - K-means ARI: {calculate_ari(breast_cancer_labels, breast_cancer_kmeans_labels)}")
print(f"Abalone Dataset - K-means ARI: {calculate_ari(abalone_labels, abalone_kmeans_labels)}")
print(f"Ecoli Dataset - K-means ARI: {calculate_ari(ecoli_labels, ecoli_kmeans_labels)}")

# DBSCAN
print(f"\nWine Quality Dataset - DBSCAN ARI: {calculate_ari(wine_labels, wine_dbscan_labels)}")
print(f"Breast Cancer Wisconsin Dataset - DBSCAN ARI: {calculate_ari(breast_cancer_labels, breast_cancer_dbscan_labels)}")
print(f"Abalone Dataset - DBSCAN ARI: {calculate_ari(abalone_labels, abalone_dbscan_labels)}")
print(f"Ecoli Dataset - DBSCAN ARI: {calculate_ari(ecoli_labels, ecoli_dbscan_labels)}")

# HDBSCAN
print(f"\nWine Quality Dataset - HDBSCAN ARI: {calculate_ari(wine_labels, wine_hdbscan_labels)}")
print(f"Breast Cancer Wisconsin Dataset - HDBSCAN ARI: {calculate_ari(breast_cancer_labels, breast_cancer_hdbscan_labels)}")
print(f"Abalone Dataset - HDBSCAN ARI: {calculate_ari(abalone_labels, abalone_hdbscan_labels)}")
print(f"Ecoli Dataset - HDBSCAN ARI: {calculate_ari(ecoli_labels, ecoli_hdbscan_labels)}")


Adjusted Rand Index (ARI):
Wine Quality Dataset - K-means ARI: 0.03893620193092464
Breast Cancer Wisconsin Dataset - K-means ARI: 0.7177694814483356
Abalone Dataset - K-means ARI: 0.0763969650221238
Ecoli Dataset - K-means ARI: 0.2975447755391641

Wine Quality Dataset - DBSCAN ARI: 0.003301615359318296
Breast Cancer Wisconsin Dataset - DBSCAN ARI: 0.0
Abalone Dataset - DBSCAN ARI: 0.04832540214261044
Ecoli Dataset - DBSCAN ARI: -0.08223012579256113

Wine Quality Dataset - HDBSCAN ARI: 0.005406300562272674
Breast Cancer Wisconsin Dataset - HDBSCAN ARI: 0.0
Abalone Dataset - HDBSCAN ARI: 0.04345287337663233
Ecoli Dataset - HDBSCAN ARI: 0.3555653194337105


In [12]:
from scipy.spatial.distance import pdist, cdist, euclidean
from sklearn.metrics import pairwise_distances

def dunn_index(X, labels):
    unique_labels = np.unique(labels)
    distances = pairwise_distances(X)
    inter_cluster_distances = np.inf
    intra_cluster_distances = 0

    for label in unique_labels:
        if label == -1:  # Ignore noise if present
            continue
        cluster_points = X[labels == label]
        intra_cluster_distances = max(intra_cluster_distances, np.max(pdist(cluster_points)))

    for i, label1 in enumerate(unique_labels[:-1]):
        if label1 == -1:
            continue
        cluster_points1 = X[labels == label1]
        for label2 in unique_labels[i + 1:]:
            if label2 == -1:
                continue
            cluster_points2 = X[labels == label2]
            inter_cluster_distances = min(inter_cluster_distances, np.min(cdist(cluster_points1, cluster_points2)))

    return inter_cluster_distances / intra_cluster_distances

print("Dunn Index:")

# K-means
print(f"\nWine Quality Dataset - K-means Dunn Index: {dunn_index(wine_clean_scaled.values, wine_kmeans_labels)}")
print(f"Breast Cancer Wisconsin Dataset - K-means Dunn Index: {dunn_index(breast_cancer_clean_scaled.values, breast_cancer_kmeans_labels)}")
print(f"Abalone Dataset - K-means Dunn Index: {dunn_index(abalone_clean_scaled.values, abalone_kmeans_labels)}")
print(f"Ecoli Dataset - K-means Dunn Index: {dunn_index(ecoli_clean_scaled.values, ecoli_kmeans_labels)}")

# DBSCAN
print(f"\nWine Quality Dataset - DBSCAN Dunn Index: {dunn_index(wine_clean_scaled.values, wine_dbscan_labels)}")
#print(f"Breast Cancer Wisconsin Dataset - DBSCAN Dunn Index: {dunn_index(breast_cancer_clean_scaled.values, breast_cancer_dbscan_labels)}")
print(f"Abalone Dataset - DBSCAN Dunn Index: {dunn_index(abalone_clean_scaled.values, abalone_dbscan_labels)}")
print(f"Ecoli Dataset - DBSCAN Dunn Index: {dunn_index(ecoli_clean_scaled.values, ecoli_dbscan_labels)}")

# HDBSCAN
print(f"\nWine Quality Dataset - HDBSCAN Dunn Index: {dunn_index(wine_clean_scaled.values, wine_hdbscan_labels)}")
#print(f"Breast Cancer Wisconsin Dataset - HDBSCAN Dunn Index: {dunn_index(breast_cancer_clean_scaled.values, breast_cancer_hdbscan_labels)}")
print(f"Abalone Dataset - HDBSCAN Dunn Index: {dunn_index(abalone_clean_scaled.values, abalone_hdbscan_labels)}")
print(f"Ecoli Dataset - HDBSCAN Dunn Index: {dunn_index(ecoli_clean_scaled.values, ecoli_hdbscan_labels)}")


Dunn Index:

Wine Quality Dataset - K-means Dunn Index: 0.02692106105694687
Breast Cancer Wisconsin Dataset - K-means Dunn Index: 0.14988308874286074
Abalone Dataset - K-means Dunn Index: 0.0260619103607442
Ecoli Dataset - K-means Dunn Index: 0.07329793799637103

Wine Quality Dataset - DBSCAN Dunn Index: 2.047444765512756
Abalone Dataset - DBSCAN Dunn Index: 0.027437258833636152
Ecoli Dataset - DBSCAN Dunn Index: 0.2778838248594125

Wine Quality Dataset - HDBSCAN Dunn Index: 0.21816354266669666
Abalone Dataset - HDBSCAN Dunn Index: 0.10606064692898444
Ecoli Dataset - HDBSCAN Dunn Index: 0.2489604503632509


In [11]:
from sklearn.metrics import rand_score, fowlkes_mallows_score

def calculate_external_indices(true_labels, predicted_labels):
    rand_index = rand_score(true_labels, predicted_labels)
    fowlkes_mallows = fowlkes_mallows_score(true_labels, predicted_labels)
    return rand_index, fowlkes_mallows

print("External Validation Indices:")

# K-means
wine_rand, wine_fm = calculate_external_indices(wine_labels, wine_kmeans_labels)
breast_cancer_rand, breast_cancer_fm = calculate_external_indices(breast_cancer_labels, breast_cancer_kmeans_labels)
abalone_rand, abalone_fm = calculate_external_indices(abalone_labels, abalone_kmeans_labels)
ecoli_rand, ecoli_fm = calculate_external_indices(ecoli_labels, ecoli_kmeans_labels)

print(f"Wine Quality Dataset - K-means Rand Index: {wine_rand}, Fowlkes-Mallows: {wine_fm}")
print(f"Breast Cancer Wisconsin Dataset - K-means Rand Index: {breast_cancer_rand}, Fowlkes-Mallows: {breast_cancer_fm}")
print(f"Abalone Dataset - K-means Rand Index: {abalone_rand}, Fowlkes-Mallows: {abalone_fm}")
print(f"Ecoli Dataset - K-means Rand Index: {ecoli_rand}, Fowlkes-Mallows: {ecoli_fm}")

# DBSCAN
wine_rand, wine_fm = calculate_external_indices(wine_labels, wine_dbscan_labels)
breast_cancer_rand, breast_cancer_fm = calculate_external_indices(breast_cancer_labels, breast_cancer_dbscan_labels)
abalone_rand, abalone_fm = calculate_external_indices(abalone_labels, abalone_dbscan_labels)
ecoli_rand, ecoli_fm = calculate_external_indices(ecoli_labels, ecoli_dbscan_labels)

print(f"Wine Quality Dataset - DBSCAN Rand Index: {wine_rand}, Fowlkes-Mallows: {wine_fm}")
print(f"Breast Cancer Wisconsin Dataset - DBSCAN Rand Index: {breast_cancer_rand}, Fowlkes-Mallows: {breast_cancer_fm}")
print(f"Abalone Dataset - DBSCAN Rand Index: {abalone_rand}, Fowlkes-Mallows: {abalone_fm}")
print(f"Ecoli Dataset - DBSCAN Rand Index: {ecoli_rand}, Fowlkes-Mallows: {ecoli_fm}")

# HDBSCAN
wine_rand, wine_fm = calculate_external_indices(wine_labels, wine_hdbscan_labels)
breast_cancer_rand, breast_cancer_fm = calculate_external_indices(breast_cancer_labels, breast_cancer_hdbscan_labels)
abalone_rand, abalone_fm = calculate_external_indices(abalone_labels, abalone_hdbscan_labels)
ecoli_rand, ecoli_fm = calculate_external_indices(ecoli_labels, ecoli_hdbscan_labels)

print(f"Wine Quality Dataset - HDBSCAN Rand Index: {wine_rand}, Fowlkes-Mallows: {wine_fm}")
print(f"Breast Cancer Wisconsin Dataset - HDBSCAN Rand Index: {breast_cancer_rand}, Fowlkes-Mallows: {breast_cancer_fm}")
print(f"Abalone Dataset - HDBSCAN Rand Index: {abalone_rand}, Fowlkes-Mallows: {abalone_fm}")
print(f"Ecoli Dataset - HDBSCAN Rand Index: {ecoli_rand}, Fowlkes-Mallows: {ecoli_fm}")

External Validation Indices:
Wine Quality Dataset - K-means Rand Index: 0.6089259185933408, Fowlkes-Mallows: 0.27886636864289943
Breast Cancer Wisconsin Dataset - K-means Rand Index: 0.8688657392757237, Fowlkes-Mallows: 0.8964614867344495
Abalone Dataset - K-means Rand Index: 0.6504764143748539, Fowlkes-Mallows: 0.2745688792344589
Ecoli Dataset - K-means Rand Index: 0.7501125040180007, Fowlkes-Mallows: 0.4639626250376742
Wine Quality Dataset - DBSCAN Rand Index: 0.37662746989208423, Fowlkes-Mallows: 0.5844624497319555
Breast Cancer Wisconsin Dataset - DBSCAN Rand Index: 0.6278622330797564, Fowlkes-Mallows: 0.7923775823934928
Abalone Dataset - DBSCAN Rand Index: 0.691381440759912, Fowlkes-Mallows: 0.22411865239295756
Ecoli Dataset - DBSCAN Rand Index: 0.3458266366655952, Fowlkes-Mallows: 0.42894344322343814
Wine Quality Dataset - HDBSCAN Rand Index: 0.4447444830771625, Fowlkes-Mallows: 0.5068671366055877
Breast Cancer Wisconsin Dataset - HDBSCAN Rand Index: 0.6278622330797564, Fowlkes-M